#Challenge

In [1]:
# If additional packages are needed but are not installed by default, uncomment the last two lines of this cell
# and replace <package list> with a list of additional packages.
# This will ensure the notebook has all the dependencies and works everywhere

#import sys
#!{sys.executable} -m pip install <package list>

In [2]:
import sys
!{sys.executable} -m pip install wget
!{sys.executable} -m pip install shutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=1763066cc55f119b09bca206682efe710a3aef735ace9be80323819617490790
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [3]:
# Libraries
import pandas as pd
import os
from shutil import unpack_archive
pd.set_option("display.max_columns", 101)

## Dataset

In [4]:
# Run this cell block to download and extract image data
!wget 'https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/8c7bc0c59ee6bcdb3646a1997606a9d0/test.zip'
!wget 'https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/ffe8804da24b94ee410a8730ba297dfc/train_0.zip'
!wget 'https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/5460fcc87e2d7f7e19c17f719e0df00a/train_1.zip'

print('Extracting Train Dataset : Label 1')
unpack_archive('train_1.zip', '')

print('Extracting Train Dataset : Label 0')
unpack_archive('train_0.zip', '')

print('Extracting Test Dataset')
unpack_archive('test.zip', '')

print('Dataset Extracted')


os.remove('train_1.zip')
os.remove('train_0.zip')
os.remove('test.zip')
print('Extra Files Removed')

--2022-07-14 18:53:15--  https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/8c7bc0c59ee6bcdb3646a1997606a9d0/test.zip
Resolving hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)... 52.216.42.153
Connecting to hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)|52.216.42.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58948996 (56M) [application/zip]
Saving to: ‘test.zip’

test.zip            100%[===================>]  56.22M  12.7MB/s    in 5.7s    

2022-07-14 18:53:22 (9.93 MB/s) - ‘test.zip’ saved [58948996/58948996]

--2022-07-14 18:53:22--  https://hr-projects-assets-prod.s3.amazonaws.com/1iaanii247i/ffe8804da24b94ee410a8730ba297dfc/train_0.zip
Resolving hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)... 52.217.80.12
Connecting to hr-projects-assets-prod.s3.amazonaws.com (hr-projects-assets-prod.s3.amazonaws.com)|52.217.80.12|:443... connec

## Data Description

Column | Description
:---|:---
`file` | Image name
`label` | Target Label (1/0)

In [5]:
label = pd.read_csv("/content/train_labels.csv")
label.head()

,file,label
0,img_44733.jpg,1.0
1,img_72999.jpg,1.0
2,img_25094.jpg,1.0
3,img_69092.jpg,1.0
4,img_92629.jpg,1.0


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      validation_split=0.2
      )
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(300,300),
    batch_size = 128,
    class_mode = "binary",
    subset="training"
)
validation_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(300,300),
    batch_size = 128,
    class_mode = "binary",
    subset="validation"
)

Found 3213 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


## Machine Learning

Build a neural network that can classify the driver’s behaviour.
- **The model's performance will be evaluated on the basis of accuracy.**

In [21]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True
callback = myCallback()

In [32]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(8, (3,3), activation="relu", input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(16, (3,3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(16, (3,3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation="relu"),
                                    tf.keras.layers.Dense(64, activation="relu"),
                                    tf.keras.layers.Dense(1, activation="sigmoid")
                                    ])

In [33]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 298, 298, 8)       224       
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 149, 149, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 147, 147, 16)      1168      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 73, 73, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 71, 71, 16)        2320      
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 35, 35, 16)      

In [34]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics = ["accuracy"])

In [35]:
history = model.fit(train_generator, epochs=25, validation_data=validation_generator,verbose = 1, callbacks = [callback])

Epoch 1/25
26/26 [==============================] - 94s 4s/step - loss: 0.6908 - accuracy: 0.5313 - val_loss: 0.6842 - val_accuracy: 0.5549
Epoch 2/25
26/26 [==============================] - 90s 3s/step - loss: 0.6875 - accuracy: 0.5521 - val_loss: 0.6751 - val_accuracy: 0.6022
Epoch 3/25
26/26 [==============================] - 90s 3s/step - loss: 0.6805 - accuracy: 0.5736 - val_loss: 0.6779 - val_accuracy: 0.5673
Epoch 4/25
26/26 [==============================] - 90s 3s/step - loss: 0.6753 - accuracy: 0.5854 - val_loss: 0.6631 - val_accuracy: 0.6097
Epoch 5/25
26/26 [==============================] - 88s 3s/step - loss: 0.6701 - accuracy: 0.5851 - val_loss: 0.6570 - val_accuracy: 0.6247
Epoch 6/25
26/26 [==============================] - 89s 3s/step - loss: 0.6655 - accuracy: 0.6060 - val_loss: 0.6620 - val_accuracy: 0.6035
Epoch 7/25
26/26 [==============================] - 89s 3s/step - loss: 0.6734 - accuracy: 0.5857 - val_loss: 0.6658 - val_accuracy: 0.6022
Epoch 8/25
26/26 [==

> #### Task:
- **Submit the predictions on the test dataset using your optimized model** <br/>
    For each image in the test folder, predict the value of the label (0/1). You should submit a CSV file with a header row and one row per test entry. 

The file (`submissions.csv`) should have exactly 2 columns:
   - **file**
   - **label**
   

In [36]:
from keras.preprocessing import image
import numpy as np

test_dir = os.path.join('/content/test')
test_names = os.listdir(test_dir)

test_list = []

# predicting images

for name in test_names:
  try:
    path = '/content/test/' + name
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = x / 255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    result = model.predict(images, batch_size=10)
  except:
    print(f"{name} not found or not a image file")
  else:
    if result[0]>0.5:
      test_list.append([name,1])
    else:
      test_list.append([name,0])

.DS_Store not found or not a image file


In [37]:
submission_df = pd.DataFrame(test_list)

In [38]:
#Submission
submission_df.to_csv('submissions.csv',index=False)